In [146]:
import torchtext
from torchtext.data.utils import get_tokenizer
from sklearn.model_selection import train_test_split
import re
from cleantext import clean
import os
from nltk.tokenize import TweetTokenizer

In [175]:
# Preprocess the data 

tweet_tokenizer = TweetTokenizer()
tokenizer = get_tokenizer('basic_english')

def replace_dates(text):
        date_format_a = re.sub(r'\d{1,2}/\d{1,2}/\d{2,4}', ' <DATE> ', text)
        date_format_b = re.sub(
            r'[A-Za-z]{2,8}\s\d{1,2},?\s\d {4}', ' <DATE> ', date_format_a)
        date_format_c = re.sub(
            r'\d{2} [A-Z][a-z]{2,8} \d{4}', ' <DATE> ', date_format_b)
        return date_format_c

def replace_concurrent_punctuation(text):
    # replace concurrent punctuation with single punctuation
    return re.sub(r'(!|"|\#|\$|%|&|\'|\(|\)|\*|\+|,|-|\.|\/|:|;|<|=|>|\?|@|\[|\\|\]|\^|_|‘|\{|\||\}|~){2,}', r' ', text)

def replace_hash_tags(text):
        return re.sub(r'(\s|^)#(\w+)', ' <HASHTAG> ', text)

def remove_special_characters(text):
        # remove special characters other than punctuation
        return re.sub(r'[^A-Za-z0-9\s\.\,\!\?\'\"\:\;]', ' ', text)

def remove_extra_spaces(text):
        return re.sub(r'\s{2,}', ' ', text)

def replace_hyphenated_words(text):
        # replace hyphenated words with words seperated by space
        return re.sub(r'(\w+)-(\w+)', r'\1 \2', text)

def read_data(filename, n_lines):
    with open(filename, 'r') as f:
        lines = []
        for _ in range(n_lines):
            line = f.readline().strip()
            line = re.sub(r'<|>', ' ', line)
            line = replace_dates(line)
            line = replace_hyphenated_words(line)
            line = replace_hash_tags(line)
            # remove < and > from the text
            line = clean(line, no_emoji=True,
                         no_urls=True,
                         no_emails=True,
                         no_phone_numbers=True,
                         no_currency_symbols=True,           
                         replace_with_url=" <URL> ",
                         replace_with_email=" <EMAIL> ",
                         replace_with_phone_number=" <PHONE> ",
                         replace_with_currency_symbol=" <CURRENCY> ",
                         lower=True)
            line = remove_special_characters(line)
            #line = replace_concurrent_punctuation(line)
            line = clean(line,no_numbers=True,no_digits=True,no_punct=True, replace_with_number=" <NUMBER> ",replace_with_digit=" ",replace_with_punct="")
            line = "<BEGIN> " + line + " <END>"
            line = remove_extra_spaces(line)
            tokens=tokenizer(line)
            if len(tokens)>1:
                lines.append(tokens)
    return lines


def save_data(filename, lines):
    # Save the data to a file
    with open(filename, 'w')as f:
        for line in lines:
            line = ' '.join(line)
            f.write(line.strip()+'\n')




In [280]:
if not os.path.exists('./processed_data'):
    os.mkdir('processed_data')

data = read_data('data/alternate/L3Cube-HingCorpus_roman/R11_final_data/concatenated_train_final_shuffled.txt',20000)
train,valid = train_test_split(data, test_size=0.3, random_state=42)
valid,test=train_test_split(valid, test_size=0.5, random_state=42)
#print(train[1:100])
save_data('processed_data/train.txt', train)
save_data('processed_data/valid.txt', valid)
save_data('processed_data/test.txt', test)

In [ ]:
#---------------------------------------------------------------------

In [202]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [281]:
class L3CubeDataset(Dataset):
    def __init__(self,filename,vocab=None,ngram=5):
        data = self.read_data(filename)
        if vocab is None:
            self.vocab, self.ind2vocab = self.build_vocab(data)
        else:
            self.vocab = vocab
            self.ind2vocab = {v:k for k,v in vocab.items()}
        self.n = ngram
        self.x,self.y = self.__create_dataset(data)
        
    def get_vocab(self):
        return self.vocab

    def read_data(self,filename):
        lines = []
        with open(filename, 'r') as f:
            for line in f.readlines():
                lines.append(line.strip().split(' '))
        return lines

    def build_vocab(self,data):
        word_set = set()
        for line in data:
            for word in line:
                if word not in word_set:
                    word_set.add(word)
        # sort the vocab
        word_list = sorted(list(word_set))
        vocab_dict = {"<unk>":0}
        for i,word in enumerate(word_list):
            vocab_dict[word]=i+1
        ind2word = {v:k for k,v in vocab_dict.items()}
        return vocab_dict, ind2word
    
    def get_ngram(self, tokens):
        n =self.n
        ngram = []
        if len(tokens) == 0:
            return None
        tokens = ["<begin>" for _ in range(n-2)] + tokens
        for i in range(len(tokens)-n+1):
            ngram.append(tokens[i:i+n])
        return ngram
    
    def __get_seq(self, tokens):
        vec= []
        for word in tokens:
            if word in self.vocab:
                vec.append(self.vocab[word])
            else:
                vec.append(self.vocab["<unk>"])
        return vec

    def __create_dataset(self, data):
        x = []
        y= []
        ngrams = []
        for line in data:
            ngrams.extend(self.get_ngram(line))
        
        for ngram in ngrams:
            x_tokens = ngram[:-1]
            y_tokens = ngram[1:]
            x.append(self.__get_seq(x_tokens))
            y.append(self.__get_seq(y_tokens))
        return torch.LongTensor(x),torch.LongTensor(y)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]
    
    def get_dataloader(self, batch_size,shuffle=True):
        return DataLoader(self, batch_size=batch_size, shuffle=shuffle,drop_last=True)
    

In [282]:
train_dataset = L3CubeDataset('processed_data/train.txt')
validation_dataset = L3CubeDataset('processed_data/valid.txt',vocab=train_dataset.get_vocab())

In [283]:
import json
json.dump(train_dataset.get_vocab(),open('vocab.json','w'))
len(train_dataset.get_vocab())

24011

In [284]:
vocab_dict = train_dataset.get_vocab()
for word in vocab_dict:
    if len(word) > 10:
        print(word)



aajtakabpnews
aapdisappointing
aayegaseekh
abhicrackers
accidentally
accomplished
accountability
accountable
achanakmaine
achievement
achievements
acknowledge
acknowledged
acquaintance
additionally
adhikariyon
administration
administrative
adversities
advertisement
advicekabhi
aerodynamic
affiliations
afghanistan
afternoonpartly
agricultural
agriculture
alaipayuthey
alhamdulillah
allegations
alnabilirdi
alreadythen
alternative
alternatives
amavaaskhairiyat
anniversary
announcement
anthropocene
anticipated
anyonemaine
apocalptica
appearances
appeasement
application
appointment
appreciated
appreciating
appreciation
apprhension
appropriate
appropriation
approximately
argumentative
arrangement
artificially
assalamualaikumagar
assassinated
assignments
association
audiencehas
authenticated
authoritarian
authorities
automatically
availability
ayengekyounder
ayudarlosmejor
baatayeithey
badhiyapure
bajengeeeeeee
balochistan
banaobeuske
bandaofficial
banugitujhe
bardhamaner
baseerullah
beautific

In [285]:
print(train_dataset.x[2],train_dataset.y[2])

tensor([    1,     1, 11688, 12520]) tensor([    1, 11688, 12520, 12030])


In [267]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.nn.utils import clip_grad_norm_
from tqdm import tqdm
import numpy as np

In [293]:
class GramNet(nn.Module):
    def __init__(self,vocab_size, n_hidden=256, n_layers=4,embedding_dim=200, dropout=0.2, lr=0.001,device='cuda'):
        super().__init__()
        self.dropout = dropout
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        self.device = device
        self.vocab_size = vocab_size
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, n_hidden, n_layers, dropout=dropout,batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(n_hidden, vocab_size)      
    
    def forward(self, x, hidden):
        embedded = self.embedding(x)     
        lstm_output, hidden = self.rnn(embedded, hidden)
        out = self.dropout(lstm_output)
        out = out.reshape(-1, self.n_hidden) 
        out = self.fc(out)
        return out, hidden
    
    def __init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().to(device),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().to(device))
        return hidden
    
    def accuracy(self,true, pred):
        true = np.array(true)
        pred = np.array(pred)
        num_correct = sum(true == pred)
        num_total = len(true)
        return num_correct / num_total


    def run_training(self,train_dataset,valid_dataset, epochs=10, batch_size=32, clip = 1,print_every=1):
        device = self.device
        if str(device) == 'cpu':
            print("Training only supported in GPU environment")
            return
        torch.cuda.empty_cache()
        self.to(device)
        train_loader = train_dataset.get_dataloader(batch_size)
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        criterion = nn.CrossEntropyLoss()
        self.train()
        for epoch in range(epochs):
            hidden = self.__init_hidden(batch_size)
            for i, (x, y) in enumerate(train_loader):
                hidden = tuple([each.data for each in hidden])
                x, y = x.to(device), y.to(device)
                output, hidden = self.forward(x, hidden)
                loss = criterion(output, y.view(-1))
                optimizer.zero_grad()
                loss.backward()
                clip_grad_norm_(self.parameters(), clip)
                optimizer.step()
            if i % print_every == 0:
                acc,_ = self.evaluate(train_dataset)
                acc2,_ = self.evaluate(valid_dataset)
                self.train()
                print("Epoch: {}/{}".format(epoch+1, epochs),
#                       "Step: {}".format(i),
                      "Loss: {}".format(loss.item()),
                      "Training Accuracy: {}".format(acc),
                      "Validation Accuracy: {}".format(acc2))
                    
    def evaluate(self, dataset, batch_size=32):
        device = self.device
        self.to(device)
        self.eval()
        loader = dataset.get_dataloader(batch_size)
        hidden = self.__init_hidden(batch_size)
        preds = []
        trues = []
        for i, (x, y) in enumerate(loader):
            hidden = tuple([each.data for each in hidden])
            x, y = x.to(device), y
            output, hidden = self.forward(x, hidden)
            preds.extend(output.argmax(dim=1).cpu().numpy())
            trues.extend(y.view(-1).numpy())
        accuracy = self.accuracy(trues, preds)
        return accuracy, preds

In [294]:
net = GramNet(len(train_dataset.get_vocab()))
net.cuda()
print(net)

GramNet(
  (embedding): Embedding(24011, 200)
  (rnn): LSTM(200, 256, num_layers=4, batch_first=True, dropout=0.2)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=256, out_features=24011, bias=True)
)


In [295]:
net.run_training(train_dataset,validation_dataset, epochs=2, batch_size=256, clip = 1,print_every=200)

1944it [00:03, 527.54it/s]
7it [00:04,  2.22it/s]

Epoch: 1/2 Step: 0 Loss: 10.087209701538086 Validation Accuracy: 0.0013904963991769547


1944it [00:03, 496.54it/s]
210it [00:12,  5.58it/s]

Epoch: 1/2 Step: 200 Loss: 7.362147331237793 Validation Accuracy: 0.07203655478395062


1944it [00:03, 564.02it/s]
407it [00:19,  6.12it/s]

...

1944it [00:04, 435.43it/s]
809it [00:38,  4.69it/s]

Epoch: 2/2 Step: 800 Loss: 6.546810626983643 Validation Accuracy: 0.11832079475308642


1944it [00:04, 408.45it/s]
1009it [00:47,  4.28it/s]

Epoch: 2/2 Step: 1000 Loss: 6.3822126388549805 Validation Accuracy: 0.1206516846707819


1122it [00:49, 22.50it/s]


In [ ]:
torch.save(net.state_dict(), './saved_models/model_1.pt')

In [ ]:
net2 = WordLSTM(len(train_dataset.get_vocab()))
net2.load_state_dict(torch.load('./saved_models/model_1.pt'))

In [259]:
import math

In [ ]:
def get_perp(prob,n):
  p = math.exp(prob*(1/n))
  return 1/p

In [258]:
def predict_prob(net, tkn, trg, h=None): 
    # tensor inputs
    if tkn in token2int:
      x = np.array([[token2int[tkn]]])
    else:
      x = np.array([[token2int['UNK']]])

    inputs = torch.from_numpy(x)

    # push to GPU
    inputs = inputs.cuda()

    # detach hidden state from history
    h = tuple([each.data for each in h])

    # get the output of the model
    out, h = net(inputs, h)

    # get the token probabilities
    p = F.softmax(out, dim=1).data

    p = p.cpu()

    p = p.numpy()
    p = p.reshape(p.shape[1],)

    if trg in token2int:
        prob = p[token2int[trg]]
    else:
        prob = p[token2int['UNK']]

    return h, prob
# function to calculate perplexity


In [ ]:
def get_prob(net, x, y):

    # push to GPU
    net.cuda()

    net.eval()

    # batch size is 1
    h = net.init_hidden(1)

    N = len(x)
    tmp = 1

    for i, gram in enumerate(x):
        h, prob = predict_prob(net, gram, y[i], h)
        if (i==0): 
          if (gram in word_counter): tmp*= word_counter[gram]/vocab_size
          else: tmp*= ind/vocab_size   
        else:
          tmp *= prob
        # print(gram,prob,tmp)

    return math.log(tmp)